Version 1.6, 14 May 2024

STORM Track dataset details: 

https://www.nature.com/articles/s41597-020-0381-2

https://doi.org/10.4121/uuid:82c1dc0d-5485-43d8-901a-ce7f26cda35d

https://data.4tu.nl/articles/STORM_IBTrACS_present_climate_synthetic_tropical_cyclone_tracks/12706085?file=24876980

The 10,000 year TC STORM dataset, based on the present climate, is publicly accessible and can be found on the 4TU.Centre for Research Data repository (https://doi.org/10.4121/uuid:82c1dc0d-5485-43d8-901a-ce7f26cda35d). The dataset is split in separate files per basin, with each .txt-file containing 1,000 years of simulations (i.e. 10 files per basin). Each .txt-file consists of a series of arrays, with each array being a single time step (3-hourly) for a synthetic TC. The colmns of the arrays are given in this table:

Image
Bloemendaal, Nadia; Haigh, I.D. (Ivan); de Moel, H. (Hans); Muis, S; Haarsma, R.J. (Reindert) et. al. (2022): STORM IBTrACS present climate synthetic tropical cyclone tracks. Version 4. 4TU.ResearchData. dataset. https://doi.org/10.4121/12706085.v4

Bloemendaal, Nadia; de Moel, H. (Hans); Martinez, Andrew B.; Muis, S. (Sanne); Haigh, I.D. (Ivan) et. al. (2023): STORM Climate Change synthetic tropical cyclone tracks. Version 2. 4TU.ResearchData. dataset. https://doi.org/10.4121/14237678.v2

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import cartopy.crs as ccrs

import cheer_utils
import importlib
importlib.reload(cheer_utils)
print(cheer_utils.__doc__)


from scipy import interpolate as sci
#from scipy.interpolate import RBFInterpolator

# pd.options.mode.copy_on_write = True

## https://visibleearth.nasa.gov/collection/1484/blue-marble
## https://neo.gsfc.nasa.gov/view.php?datasetId=BlueMarbleNG
#os.environ["CARTOPY_USER_BACKGROUNDS"] = "/home/bblanton/GitHub/BrianOBlanton/CHEER-STORM/static"


In [ ]:
# define some plotting vars
makeplots=True
writeoutputfiles=False
# marker size
sz=20

## Load IBTRaCS
## converts units to MKS
### Parameters to LoadIBTrACS:

- minyear: delete tracks < minyear
- maxyear: delete tracks > maxyear

In [ ]:
df_ib=cheer_utils.LoadIBTrACS(minyear=1980)
df_ib

In [ ]:
# show complete track for storm with largest RMW
# print(df_ib['RMW'].max())
# df_ib.loc[df_ib.RMW.idxmax()]

In [ ]:
# further screening of df
#df_ib=df_ib.loc[(df_ib['NATURE']=='TS')]
# & (df['Min_pres'] <= 980)]

## load STORM dataset
## default is to read first 1000 yrs of present climate
### Parameters to LoadSTORMtracks:
- basin='NA' 
- ensnum=0
- climate='current'
- model='present'
- version='_V4'
- nyears=None
- startingyear=None

In [ ]:
nyears="all"
#nyears=100
ensnum=0
df_st=cheer_utils.LoadSTORMtracks(ensnum=ensnum) #,nyears=nyears)
IDX=np.unique(df_st.index).astype(int)
print(IDX)
# df_st=df_st.loc[(df_st.index == IDX[0]) | (df_st.index == IDX[1])] 
#df_st=df_st.loc[df_st.RMW.idxmax()]
#df_st=df_st.iloc[0:5]
df_st

In [ ]:
setnums=10
setnums=np.arange(setnums)
print(type(setnums))
setnums
ny=2017-1979
i=3
(i-1)*ny

### Load PEPC data

In [ ]:
path2pepcfiles="PEPC_Tracks/reanal_100"
#setnums=range(1,2)
setnums='all'
df_pc=cheer_utils.LoadPEPC(path2pepcfiles,setnums=setnums) # loads first set
df_pc

## Add distance to sites, Delta P, and Holland-B

In [ ]:
df_ib['dist2nc']=np.sqrt(np.square(df_ib['Longitude']-nc_circ['lo']) + np.square(df_ib['Latitude']-nc_circ['la']))
df_ib['dist2tx']=np.sqrt(np.square(df_ib['Longitude']-tx_circ['lo']) + np.square(df_ib['Latitude']-tx_circ['la']))
df_ib['HollandB']=cheer_utils.HbFromRmwLat(df_ib['RMW'],df_ib['Latitude'])
df_ib['DeltaP']=-(df_ib['Min_pres']-1024)

df_st['dist2nc']=np.sqrt(np.square(df_st['Longitude']-nc_circ['lo']) + np.square(df_st['Latitude']-nc_circ['la']))
df_st['dist2tx']=np.sqrt(np.square(df_st['Longitude']-tx_circ['lo']) + np.square(df_st['Latitude']-tx_circ['la']))
df_st['HollandB']=cheer_utils.HbFromRmwLat(df_st['RMW'],df_st['Latitude'])
df_st['DeltaP']=-(df_st['Min_pres']-1013)

df_pc['dist2nc']=np.sqrt(np.square(df_pc['Longitude']-nc_circ['lo']) + np.square(df_pc['Latitude']-nc_circ['la']))
df_pc['dist2tx']=np.sqrt(np.square(df_pc['Longitude']-tx_circ['lo']) + np.square(df_pc['Latitude']-tx_circ['la']))
#df_pc['HollandB']=cheer_utils.HbFromRmwLat(df_pc['RMW'],df_pc['Latitude'])
#df_pc['DeltaP']=-(df_pc['Min_pres']-1013)

In [ ]:
# extract genesis locations
df_ib_g=cheer_utils.get_genesis_locations(df_ib)   
df_st_g=cheer_utils.get_genesis_locations(df_st)   
df_pc_g=cheer_utils.get_genesis_locations(df_pc)   

In [ ]:
# generate a grid for histogram2d
lo0=-100
lo1=0
la0=0
la1=70
dx,dy=5,5
lon_bins=np.arange(lo0,lo1,dx)
lat_bins=np.arange(la0,la1,dy)
grid_dict={'lo0':lo0, 'lo1':lo1,  'la0':la0, 'la1':la1, 'dx':dx, 'dy':dy, 'lon_bins':lon_bins, 'lat_bins':lat_bins}

In [ ]:
def spatial_plot(data, ax, cmap, vmin, vmax, cbstr, tstr):

    kwargs={"transform":ccrs.PlateCarree(), 
            "shading":'flat', "edgecolor":'k', "linewidth":.01}
    
    axm=ax.pcolormesh(lon_bins, lat_bins, data, cmap=cmap, 
                      vmin=vmin, vmax=vmax, **kwargs)
    
    #axm=ax.pcolormesh(lon_bins-dx/2, lat_bins-dy/2, data, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(), shading='nearest')
    #axm=ax.pcolormesh(lon_bins-dx/2, lat_bins-dy/2, data,vmin=vmin, vmax=vmax,transform=ccrs.PlateCarree())

    ax.set_extent([lo0, lo1, la0, la1], crs=ccrs.PlateCarree())
    ax.grid(True)
    ax.stock_img()
    #ax[i].background_img(name='BM', resolution='low')
    cb=fig.colorbar(axm, ax=ax, shrink=0.35) #,ticks=ticks-0.5)

    cb.ax.set_ylabel(cbstr)
    ax.coastlines()
    ax.set_title(tstr)
    gl=ax.gridlines(draw_labels=True, linewidth=1, color='k', alpha=0.25, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    
    return axm, cb

In [ ]:
#get number of tracks in each cell
counts_st=cheer_utils.get_counts_per_year(grid_dict,df_st)
maxcounts_st=int(np.nanmax(counts_st))
print(f'maxcounts_st={maxcounts_st}')

counts_ib=cheer_utils.get_counts_per_year(grid_dict,df_ib)
maxcounts_ib=int(np.nanmax(counts_ib))
print(f'maxcounts_ib={maxcounts_ib}')

counts_pc=cheer_utils.get_counts_per_year(grid_dict,df_pc)
maxcounts_pc=int(np.nanmax(counts_pc))
print(f'maxcounts_pc={maxcounts_pc}')


In [ ]:
nc=np.max([maxcounts_ib,maxcounts_st,maxcounts_pc])+1

ThisCmap=cheer_utils.discrete_cmap(nc*5, 'RdBu_r')

projection=ccrs.Orthographic(central_longitude=-60,central_latitude=0)
fig, ax = plt.subplots(nrows=1,ncols=3,
                       subplot_kw={'projection': projection},
                       figsize=(24,12),
                       sharey=True)


ny=(df_ib["Year"].max()-df_ib["Year"].min())+1
tstr=f'IBTrACS, nyears={ny}'
axm, cb=spatial_plot(counts_ib,ax[0],ThisCmap, 0, nc, 'storms/yr', tstr)

ny=(df_st["Year"].max()-df_st["Year"].min())+1
tstr=f'STORM, nyears={ny}'
axm, cb=spatial_plot(counts_st,ax[1],ThisCmap, 0, nc, 'storms/yr', tstr)

ny=(df_pc["Year"].max()-df_pc["Year"].min())+1
tstr=f'PEPC, nyears={ny}'
axm, cb=spatial_plot(counts_pc,ax[2],ThisCmap, 0, nc, 'storms/yr', tstr)

# annotate with text for counts in boxes
# nx=grid_dict['lon_bins'].shape[0]
# ny=grid_dict['lat_bins'].shape[0]
# kwargs={"transform": ccrs.PlateCarree(),
#         "color":'w',
#         "fontsize":12,
#         "horizontalalignment":'center', 
#         "verticalalignment":'center'}
# for i in range(ny-1):
#     for j in range(nx-1):
#         if np.isfinite(counts_ib[i][j]):
#             ax[0].text(lon_bins[j]+dx/2,lat_bins[i]+dy/2,f'{counts_ib[i][j]:.2f}',**kwargs)
#         if np.isfinite(counts_st[i][j]):
#             ax[1].text(lon_bins[j]+dx/2,lat_bins[i]+dy/2,f'{counts_st[i][j]:.2f}',**kwargs)
#         if np.isfinite(counts_pc[i][j]):
#             ax[2].text(lon_bins[j]+dx/2,lat_bins[i]+dy/2,f'{counts_pc[i][j]:.2f}',**kwargs)


# add tracks for reference / debugging.  This takes a long time when there are many years in PEPC, STORM.
# kwargs={"transform": ccrs.PlateCarree(),
#         "color":'gray',
#         "alpha":0.5,
#         "linewidth":1}
# IDX=np.unique(df_ib.index).astype(int)   
# for i,idx in enumerate(IDX): 
#     x=df_ib.loc[df_ib.index==idx].Longitude.values
#     y=df_ib.loc[df_ib.index==idx].Latitude.values
#     ax[0].plot(x, y, **kwargs)
    
# IDX=np.unique(df_st.index).astype(int)   
# for i,idx in enumerate(IDX): 
#     x=df_st.loc[df_st.index==idx].Longitude.values
#     y=df_st.loc[df_st.index==idx].Latitude.values
#     ax[1].plot(x, y, **kwargs)

# IDX=np.unique(df_pc.index).astype(int)   
# for i,idx in enumerate(IDX): 
#     x=df_pc.loc[df_pc.index==idx].Longitude.values
#     y=df_pc.loc[df_pc.index==idx].Latitude.values
#     ax[2].plot(x, y, **kwargs)

#fig.suptitle('Annual Counts',fontsize=28,y=0.85);
#fig.save("AnnualCounts.png")

## Density plots

In [ ]:
genesis_locations=False
density=False
log10=True
normalized_count=True
cmapName='RdBu_r'

if genesis_locations:
    density_ib,      _,      _ = np.histogram2d(df_ib_g.Longitude,df_ib_g.Latitude, bins=(lon_bins,lat_bins), density=density)
    density_st,      _,      _ = np.histogram2d(df_st_g.Longitude,df_st_g.Latitude, bins=(lon_bins,lat_bins), density=density)
    density_pc, xedges, yedges = np.histogram2d(df_pc_g.Longitude,df_pc_g.Latitude, bins=(lon_bins,lat_bins), density=density)
    suptitle="Genesis Locations"
else:
    density_ib,      _,      _ = np.histogram2d(df_ib.Longitude,df_ib.Latitude, bins=(lon_bins,lat_bins), density=density)
    density_st,      _,      _ = np.histogram2d(df_st.Longitude,df_st.Latitude, bins=(lon_bins,lat_bins), density=density)
    density_pc, xedges, yedges = np.histogram2d(df_pc.Longitude,df_pc.Latitude, bins=(lon_bins,lat_bins), density=density)
    suptitle="Track Positions"

# print(density_ib.min(),density_ib.max())
# print(density_st.min(),density_st.max())
# print(density_pc.min(),density_pc.max())
nc=10

vmin=[0,0,0]
vmax=[0,0,0]

if makeplots:

    fig, ax = plt.subplots(nrows=1,ncols=3,
                           subplot_kw={'projection': ccrs.PlateCarree()},
                           figsize=(24,12), sharey=True)
  
    if density:
        
        cbstr='Point density (n/N/area)'
        
        tol=1e-4

        tempib=np.where(density_ib.T < tol, np.nan, density_ib.T)
        tempst=np.where(density_st.T < tol, np.nan, density_st.T)
        temppc=np.where(density_pc.T < tol, np.nan, density_pc.T)

        if log10:
            
            tempib=np.log10(tempib)
            tempst=np.log10(tempst)
            temppc=np.log10(temppc)
            
            vmin[0], vmax[0] = -4, -2.8
            vmin[1], vmax[1] = vmin[0], vmax[0]
            vmin[2], vmax[2] = vmin[0], vmax[0]
            nc=12
            
            cbstr=f'log10 ({cbstr})'
            
        else:
            
            vmin[0], vmax[0] = tol, 1e-3
            vmin[1], vmax[1] = vmin[0], vmax[0]
            vmin[2], vmax[2] = vmin[0], vmax[0]
            nc=10
            
    else:
        
        cbstr='Count'
        nc=10
        
        tempib=density_ib.T.copy()
        tempst=density_st.T.copy()
        temppc=density_pc.T.copy()
        
        tempib=np.where(tempib < 1, np.nan, tempib)
        tempst=np.where(tempst < 1, np.nan, tempst)
        temppc=np.where(temppc < 1, np.nan, temppc)
        
        # print(np.nanmax(tempib),np.log10(np.nanmax(tempib)),np.ceil(np.log10(np.nanmax(tempib))))
        # print(np.nanmax(tempst),np.log10(np.nanmax(tempst)),np.ceil(np.log10(np.nanmax(tempst))))
        # print(np.nanmax(temppc),np.log10(np.nanmax(temppc)),np.ceil(np.log10(np.nanmax(temppc))))

        if normalized_count:
            
            tempib=tempib/np.nanmax(tempib)
            tempst=tempst/np.nanmax(tempst)
            temppc=temppc/np.nanmax(temppc)
            cbstr=f'Normalized {cbstr}'
            vmin[0],vmax[0] = 0, 1
            vmin[1],vmax[1] = vmin[0],vmax[0]
            vmin[2],vmax[2] = vmin[0],vmax[0]

        else:
            
            # vmin0, vmax0 = 0, np.power(10,np.ceil(np.log10(np.nanmax(tempib))))
            # vmin1, vmax1 = 0, np.power(10,np.ceil(np.log10(np.nanmax(tempst))))
            # vmin2, vmax2 = 0, np.power(10,np.ceil(np.log10(np.nanmax(temppc))))
            vmin[0],vmax[0] = 0, 20
            vmin[1],vmax[1] = 0, 100            
            vmin[2],vmax[2] = 0, 100            
            
    # print(f'vmin0,vmax0={vmin[0], vmax[0]}')
    # print(f'vmin1,vmax1={vmin[1], vmax[1]}')
    # print(f'vmin2,vmax2={vmin[2] ,vmax[2]}')
    
    tstr=[f'IBTrACS \n{df_ib["Year"].min()}-{df_ib["Year"].max()}',
          f'STORM \nnyears={nyears}, ensnum={ensnum}, {int(df_st["Year"].min())}-{int(df_st["Year"].max())}',
          f'PEPC\n sets all {setnums}']
        
    ThisCmap=cheer_utils.discrete_cmap(nc, cmapName)

    axm=spatial_plot(tempib,ax[0],ThisCmap, vmin[0], vmax[0],cbstr,tstr[0])
    axm=spatial_plot(tempst,ax[1],ThisCmap, vmin[1], vmax[1],cbstr,tstr[1])
    axm=spatial_plot(temppc,ax[2],ThisCmap, vmin[2], vmax[2],cbstr,tstr[2])
    
    # hex grid
    # axm.append(ax[0].hexbin(df_gib.Longitude,df_gib.Latitude, cmap=ThisCmap, bins='log' )) # ,  vmin=vmin, vmax=vmax) 
    # axm.append(ax[1].hexbin(df_gst.Longitude,df_gst.Latitude, cmap=ThisCmap, bins='log' )) # ,  vmin=vmin, vmax=vmax) 

#     for i in range(0,3):
        
#         #ax[i].axis('equal')
#         ax[i].set_ylim([la0, la1])
#         ax[i].set_xlim([lo0, lo1])
#         ax[i].grid(True)
#         ax[i].set_aspect(1.0/np.cos(np.mean(lat_bins) * np.pi / 180.0))
#         ax[i].stock_img()
#         #ax[i].background_img(name='BM', resolution='low')
#         cb=fig.colorbar(axm[i], ax=ax[i], shrink=0.35)
#         cb.ax.set_ylabel(cbstr)
#         ax[i].coastlines()
#         ax[i].set_title(tstr[i])
#         gl=ax[i].gridlines(draw_labels=True, linewidth=1, color='k', alpha=0.5, linestyle='--')
#         gl.top_labels = False
#         gl.right_labels = False
        
    ax[0].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree(),color='k')
    ax[1].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree(),color='k')
    ax[2].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree(),color='k')
    
fig.suptitle(suptitle,fontsize=28,y=0.85);
#fig.save("Genesis_Locations.png")

## Apply "spatial" screening for study sites

In [ ]:
df_ib_nc=df_ib.loc[(df_ib['dist2nc'] <= nc_circ['rad'])] #  & (df_ib['NATURE']=='TS')]
df_st_nc=df_st.loc[(df_st['dist2nc'] <= nc_circ['rad'])]
df_pc_nc=df_pc.loc[(df_pc['dist2nc'] <= nc_circ['rad'])]

In [ ]:
importlib.reload(cheer_utils)
norm = mpl.colors.Normalize(vmin=0, vmax=100)

if makeplots:

    fig, ax = plt.subplots(nrows=1,ncols=2,
                           subplot_kw={'projection': ccrs.PlateCarree()},
                           figsize=(16,7))
    ax=ax.flatten()
    
    ax[0].stock_img()
    ax[0].coastlines()
    ax[1].stock_img()
    ax[1].coastlines()
    
#     ax[0].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree())
#     ax[1].plot(tx_circ['cirx'],tx_circ['ciry'],transform=ccrs.PlateCarree())

#     ax[0].plot(nc_circ['lo'],nc_circ['la'],marker='*',transform=ccrs.PlateCarree())
#     ax[1].plot(tx_circ['lo'],tx_circ['la'],marker='*',transform=ccrs.PlateCarree())

    for i in range(0,2):
        ax[i].axis('equal')
        ax[i].set_ylim([10, 50])
        ax[i].set_xlim([-120, 0])
        ax[i].grid(True)

cheer_utils.TrackPlot(df_ib, axx=ax[0], addcolorbar=True, norm=norm)
cheer_utils.TrackPlot(df_st, axx=ax[1], addcolorbar=True, norm=norm)

In [ ]:
print(df_ib_nc.shape)
print(df_st_nc.shape)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2,
                       subplot_kw={'projection': ccrs.PlateCarree()},
                       figsize=(16,6))
ax=ax.flatten()

ax[0].stock_img()
ax[0].coastlines()
ax[1].stock_img()
ax[1].coastlines()

ax[0].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree())
ax[1].plot(nc_circ['cirx'],nc_circ['ciry'],transform=ccrs.PlateCarree())

ax[0].plot(nc_circ['lo'],nc_circ['la'],marker='*',transform=ccrs.PlateCarree())
ax[1].plot(nc_circ['lo'],nc_circ['la'],marker='*',transform=ccrs.PlateCarree())

cmap=cheer_utils.discrete_cmap(20, 'jet')

for i in range(0,2):
    ax[i].axis('equal')
    ax[i].set_ylim([30, 40])
    ax[i].set_xlim([-82, -65])
    ax[i].grid(True)

df_ib_nc.plot.scatter(
            x='Longitude',
            y='Latitude',
            c='DeltaP',
            ax=ax[0],vmin=0,vmax=80,cmap=cmap)
df_st_nc.plot.scatter(
            x='Longitude',
            y='Latitude',
            c='DeltaP',
            ax=ax[1],vmin=0,vmax=80,cmap=cmap)

ax[0].set_title('IBTrACS')
ax[1].set_title('STORM')

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=2,
                       figsize=(12,5),sharey=True)
k='DeltaP'
i=2
axx=(df_ib_nc[k]).hist(ax=ax[0], density=True, bins=range(0,100,5))
#axx=((df_ib[k])).hist(ax=ax[0], density=True, bins=range(900,1020,10))
axx.set_ylim([0, .040])
axx.set_xlabel('[mb]')
ax[0].set_title(f'IBTrACS {k}',fontsize=20)

i=3
axx=(df_st_nc[k]).hist(ax=ax[1],density=True, bins=range(0,100,5))
#axx=((df_st['RMW'])).hist( ax=ax[1],density=True, bins=range(900,1020,10))
axx.set_ylim([0, .040])
axx.set_xlabel('[mb]')
ax[1].set_title(f'STORM {k}',fontsize=20)

In [ ]:
pd.cut(df_ib_nc.DeltaP, bins=10)

In [ ]:
#genesis  dp
fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(16,4), sharey=True) 

ax=ax.flatten()

k='DeltaP'
i=0
axx=((df_ib_g[k])).hist(ax=ax[i], density=True, bins=range(0,50,2))
#axx=((df_ib[k])).hist(ax=ax[0], density=True, bins=range(900,1020,10))
#axx.set_ylim([0, .040])
#axx.set_xlabel(k)
ax[i].set_title(k,fontsize=20)
ax[i].set_title(f'IBTrACS {k}',fontsize=20)

i=1
axx=((df_st_g[k])).hist(ax=ax[i],density=True, bins=range(0,50,2))
#axx=((df_st['RMW'])).hist( ax=ax[1],density=True, bins=range(900,1020,10))
#axx.set_ylim([0, .040])
#axx.set_xlabel(k)
ax[i].set_title(f'STORM {k}',fontsize=20)

print(df_ib_g[k].max())
print(df_st_g[k].max())


In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=2,figsize=(16,7), sharey=True) 

ax=ax.flatten()

k='DeltaP'
i=0
axx=((df_ib_nc[k])).hist(ax=ax[i], density=True, bins=range(0,140,10))
#axx=((df_ib[k])).hist(ax=ax[0], density=True, bins=range(900,1020,10))
#axx.set_ylim([0, .040])
#axx.set_xlabel(k)
ax[i].set_title(k,fontsize=20)

i=2
axx=((df_st_nc[k])).hist(ax=ax[i],density=True, bins=range(0,140,10))
#axx=((df_st['RMW'])).hist( ax=ax[1],density=True, bins=range(900,1020,10))
#axx.set_ylim([0, .040])
#axx.set_xlabel(k)
ax[i].set_title('')

print(df_ib_nc[k].max())
print(df_st_nc[k].max())

k='RMW'
i=1
axx=((df_ib_nc[k])).hist(ax=ax[i], density=True, bins=range(0,500,25))
#axx.set_ylim([0, .012])
#axx.set_xlabel(k)
ax[i].set_title(k,fontsize=20)

i=3
axx=((df_st_nc[k])).hist(ax=ax[i],density=True, bins=range(0,500,25))
#axx.set_ylim([0, .012])
#axx.set_xlabel(k)

print(df_ib_nc[k].max())
print(df_st_nc[k].max())

In [ ]:
from scipy import stats
stats.ks_2samp(df_ib_nc[np.isfinite(df_ib_nc['DeltaP'])].DeltaP, df_st_nc[np.isfinite(df_st_nc['DeltaP'])].DeltaP)

# "NC" KstestResult(statistic=0.08930426008821235, pvalue=4.81719351118903e-15, statistic_location=42.0, statistic_sign=1)
# all  KstestResult(statistic=0.1622379043492559, pvalue=0.0, statistic_location=24.0, statistic_sign=1)

In [ ]:
df_ib_nc[np.isfinite(df_ib_nc['DeltaP'])].DeltaP

In [ ]:
df_ib[df_ib["RMW"] > 35]

### interp experiments below.  not sorted out

In [ ]:

x_i, y_i = np.meshgrid(np.linspace(-100, -60, 80), np.linspace(10, 50, 80))
print(x_i.shape)
xy_i = np.concatenate([x_i.reshape(-1, 1), y_i.reshape(-1, 1)], axis=1)

# grid_x, grid_y = np.meshgrid(np.linspace(-90, -60, 20),
#                              np.linspace(10, 40, 20))
# xy_i = np.concatenate([grid_x, grid_y], axis=1)


In [ ]:
z=np.array(df_ib.Min_pres.values)
x = df_ib.Longitude.values
x = x[np.isfinite(z)] 
y = df_ib.Latitude.values
y = y[np.isfinite(z)] 
values = z[np.isfinite(z)] 

stride=10

x=x[::stride]
y=y[::stride]
values=values[::stride]

points=np.array([x,y]).T


print(x.shape)
print(values.shape)

In [ ]:
rbf = RBFInterpolator(points, values, kernel='gaussian',epsilon=2, smoothing=0.5)
z_i = rbf(xy_i).reshape(x_i.shape).T
z_i.shape

fig, ax = plt.subplots(nrows=1, ncols=1 , figsize=(6,6), dpi=100) # , sharey=True)
ax.set_aspect(1.0/np.cos(np.mean(y) * np.pi / 180.0))

axm=ax.contourf(x_i,y_i,z_i, cmap=cheer_utils.cmap,  vmin=vmin, vmax=vmax) 
#  , levels=levels, norm=norm) #
#axm=ax.scatter(x, y, 50, values, marker='o', cmap=cheer_utils.cmap,  edgecolors='k', vmin=vmin, vmax=vmax) # , norm=norm) #   
#ax.set_title(t)
fig.colorbar(axm, ax=ax, shrink=.85)
ax.plot(cheer_utils.coastline[:,0],cheer_utils.coastline[:,1],color='k',linewidth=1)
ax.set_xlim([-100-12, -40])
ax.set_ylim([5, 55])


In [ ]:
model_rbf = sci.Rbf(x,y,values, smooth=1, degree=1, function='gaussian')

In [ ]:
Z_rbf = model_rbf(x_i, y_i)
vmin=880
vmax=1025
fig, ax = plt.subplots(nrows=1, ncols=1 , figsize=(12,12), dpi=100) # , sharey=True)

axm=ax.contourf(x_i,y_i,Z_rbf, cmap=cheer_utils.cmap,  vmin=vmin, vmax=vmax) 
#  , levels=levels, norm=norm) #
ax.set_aspect(1.0/np.cos(np.mean(y) * np.pi / 180.0))
#axm=ax.scatter(x, y, 50, values, marker='o', cmap=cheer_utils.cmap,  edgecolors='k', vmin=vmin, vmax=vmax) # , norm=norm) #   
#ax.set_title(t)
fig.colorbar(axm, ax=ax, shrink=.85)
ax.plot(cheer_utils.coastline[:,0],cheer_utils.coastline[:,1],color='k',linewidth=1)
ax.set_xlim([-100-12, -40])
ax.set_ylim([5, 55])

print(np.min(values),np.max(values))
print(np.min(Z_rbf),np.max(Z_rbf))
print(Z_rbf.shape)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,
                       subplot_kw={'projection': ccrs.PlateCarree()},
                       figsize=(16,7))

ax.stock_img()
ax.coastlines()

plt.pcolor(grid_x,grid_y,z_i, )

ax.axis('equal')
ax.set_ylim([10, 50])
ax.set_xlim([-100, -50])
ax.grid(True)

In [ ]:
rng = np.random.default_rng()
points = rng.random((1000, 2))

In [ ]:
rng = np.random.default_rng()
xy = rng.random((100, 2))*4.0-2.0
z = xy[:, 0]*np.exp(-xy[:, 0]**2-xy[:, 1]**2)
edges = np.linspace(-2.0, 2.0, 101)
centers = edges[:-1] + np.diff(edges[:2])[0] / 2.
x_i, y_i = np.meshgrid(centers, centers)
x_i = x_i.reshape(-1, 1)
y_i = y_i.reshape(-1, 1)
xy_i = np.concatenate([x_i, y_i], axis=1)
rbf = RBFInterpolator(xy, z, epsilon=2)

In [ ]:
ib_yc=[]
st_yc=[]
for y in np.unique(df_ib.Year):
    i = df_ib.loc[(df_ib['Year'] == y)] 
    ib_yc.append(np.unique(i.index).shape[0])
for y in np.unique(df_st.Year):
    i = df_st.loc[(df_st['Year'] == y)] 
    st_yc.append(np.unique(i.index).shape[0])

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=1,figsize=(16,7),sharex=True, sharey=True)

ax[0].hist(ib_yc, density=True)
ax[1].hist(st_yc, density=True)

ax[0].grid(True)
ax[1].grid(True)